# Regression

In this Notebook we will examine bitcoin prices and see if we can predict the value.

https://www.kaggle.com/mczielinski/bitcoin-historical-data/data

Data under CC BY-SA 4.0 License

https://www.kaggle.com/mczielinski/bitcoin-historical-data


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import ensemble, linear_model, model_selection, preprocessing, svm
from sklearn.metrics import mean_squared_error, r2_score
from yellowbrick.regressor import PredictionError, ResidualsPlot

In [ ]:
%%time
# Resampling data from minute interval to day
bit_df = pd.read_csv('../data/coinbaseUSD_1-min_data_2014-12-01_to_2018-01-08.csv')
# Convert unix time to datetime
bit_df['date'] = pd.to_datetime(bit_df.Timestamp, unit='s')
# Reset index
bit_df = bit_df.set_index('date')
# Rename columns so easier to code
bit_df = bit_df.rename(columns={'Open':'open', 'High': 'hi', 'Low': 'lo', 
                       'Close': 'close', 'Volume_(BTC)': 'vol_btc',
                       'Volume_(Currency)': 'vol_cur', 
                       'Weighted_Price': 'wp', 'Timestamp': 'ts'})
# Resample and only use recent samples that aren't missing
bit_df = bit_df.resample('d').agg({'open': 'first', 'hi': 'max', 
    'lo': 'min', 'close': 'last', 'vol_btc': 'sum',
    'vol_cur': 'sum', 'wp': 'mean', 'ts': 'min'}).iloc[-1000:]
# drop last row as it is not complete
bit_df = bit_df.iloc[:-1]

In [ ]:
bit_df

In [ ]:
bit_df.head().T

In [ ]:
bit_df.dtypes

In [ ]:
bit_df.describe()

In [ ]:
bit_df.plot(figsize=(14,10))

In [ ]:
bit_df.close.plot(figsize=(14,10))

## Exercise: Load data
This exercise looks at predicting the size of forest fires based on meteorological data https://archive.ics.uci.edu/ml/datasets/Forest+Fires

The file is in ``../data/forestfires.csv``

* Read the data into a DataFrame
* Examine the types
* Describe the data

Attribute information:

   For more information, read [Cortez and Morais, 2007].

   1. X - x-axis spatial coordinate within the Montesinho park map: 1 to 9
   2. Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9
   3. month - month of the year: "jan" to "dec" 
   4. day - day of the week: "mon" to "sun"
   5. FFMC - FFMC index from the FWI system: 18.7 to 96.20
   6. DMC - DMC index from the FWI system: 1.1 to 291.3 
   7. DC - DC index from the FWI system: 7.9 to 860.6 
   8. ISI - ISI index from the FWI system: 0.0 to 56.10
   9. temp - temperature in Celsius degrees: 2.2 to 33.30
   10. RH - relative humidity in %: 15.0 to 100
   11. wind - wind speed in km/h: 0.40 to 9.40 
   12. rain - outside rain in mm/m2 : 0.0 to 6.4 
   13. area - the burned area of the forest (in ha): 0.00 to 1090.84 
   (this output variable is very skewed towards 0.0, thus it may make
    sense to model with the logarithm transform).

## Can we predict tomorrow's close based on today's info?
We will use a row of data for input. We will call the input X and the prediction y. This is called "supervised learning" as we will feed in both X and y to train the model.

Let's use a model called Linear Regression. This performs better if we *standardize* the data (0 mean, 1 std).

For 2 dimensions this takes the form of 

\begin{align}
y = m*x + b
\end{align}

M is the slope (or coefficient) and b is the intercept.

Let's see if we can predict the open price from the ts component.

In [ ]:
bit_df.plot(kind='scatter', x='ts', y='open', figsize=(14,10))

In [ ]:
# Create our input (X) and our labelled data (y) to train our model
X = bit_df[['ts']].iloc[:-1]  # drop last row because we don't know what future is
y = bit_df.close.shift(-1).iloc[:-1]

In [ ]:
# Train a model and predict output if it were given X
lr_model = linear_model.LinearRegression()
lr_model.fit(X, y)
pred = lr_model.predict(X)

In [ ]:
# Plot the real data, our prediction (blue), and the model from the coeffictient (green shifted)
ax = bit_df.plot(kind='scatter', x='ts', y='open', color='black', figsize=(14,10))
ax.plot(X, pred, color='blue')  # matplotlib plot
ax.plot(X, X*lr_model.coef_ + lr_model.intercept_+ 100, linestyle='--', color='green')

In [ ]:
# Vertical distance between line and point is the error. *Ordinary Least Squares* 
# regression tries to minimize the square of the distance.
mean_squared_error(y, pred)

In [ ]:
# R2 score is a measure from 0-100
# 0 - the model explains none of the variation
# 100 - 100% of the variation is explained by the model
print(r2_score(y, pred))
# Not that the .score method gives the same value
print(lr_model.score(X, y))

## Exercise: Regression

* Use linear regression to predict ``area`` from the other columns. (If you have ``object`` data columns, you can create *dummy columns* using ``pd.get_dummies``, ``pd.concat``, and ``pd.drop``)
* What is your score?

### Bonus - Visualize the errors
You can plot the actuals and the predicted values. It looks like our model does a pretty poor job

In [ ]:
# Prediction error plot from Yellowbrick
# plot of actual (blue) vs predicted (black dash)
# ideally would be around 45 degree line
fig, ax = plt.subplots(figsize=(10, 8))
err_viz = PredictionError(lr_model)
# Model is already fit
#err_viz.fit(X, y)
err_viz.score(X, y)
err_viz.poof()

In [ ]:
# plot result
y_df = pd.DataFrame(y)
y_df['pred'] = pred
y_df['err'] = y_df.pred - y_df.close
(y_df
 #.iloc[-50:]
 .plot(figsize=(14,10))
)

## Bonus Exercise: Visualize the errors
Plot y and predicted y

## Try More Features
In an attempt to get a better model we are going to use more features to make a prediction

In [ ]:
# drop last row because we don't know what future is

X = (bit_df
         .drop(['close'], axis=1)
         .iloc[:-1])
y = bit_df.close.shift(-1).iloc[:-1]
cols = X.columns

In [ ]:
# The describe method on a dataframe gives a statistical summary of the columns
X.describe()

In [ ]:
# We are going to scale the data so that volume and ts don't get more
# weight that other values
ss = preprocessing.StandardScaler()
ss.fit(X)
X = ss.transform(X)
X = pd.DataFrame(X, columns=cols)

In [ ]:
# We can now see that the data has a mean close to 0
# and a std of 1
X.describe()

In [ ]:
lr_model2 = linear_model.LinearRegression()
lr_model2.fit(X, y)
pred = lr_model2.predict(X)
lr_model2.score(X, y)

In [ ]:
# plot result
y_df = pd.DataFrame(y)
y_df['pred'] = pred
y_df['err'] = y_df.pred - y_df.close
y_df.plot(figsize=(14,10))

In [ ]:
# plot result
y_df = pd.DataFrame(y)
y_df['pred'] = pred
y_df['err'] = y_df.pred - y_df.close
y_df.iloc[-50:].plot(figsize=(14,10))

In [ ]:
# our scores get worse with recent data
lr_model2.score(X[-50:], y[-50:])

In [ ]:
lr_model2.coef_

In [ ]:
list(zip(X.columns, lr_model2.coef_))

In [ ]:
# These coefficients correspond to the columns in X
pd.DataFrame(list(zip(X.columns, lr_model2.coef_)), columns=['Feature', 'Coeff'])

In [ ]:
bit_df.plot(kind='scatter', x='wp', y='close', figsize=(14,10))

In [ ]:
bit_df.plot(kind='scatter', x='vol_cur', y='close', figsize=(14,10))

## Exercise: Regression
* Try scaling the input and using the log of the area and see if you get a better score.

* Examine the coefficients



## Training/Test Split
In fact we were cheating, predicting things that we already saw serves little purpose. The model could just memorize the data and get a perfect score. But it wouldn't *generalize* to unseen data.

To see how it will perform in the real world we will train on a portion of the data and test on a portion that it hasn't seen. 

In [ ]:
X_train, X_test, y_train, y_test = model_selection.\
    train_test_split(X, y, test_size=.3, random_state=42)

In [ ]:
lr_model2 = linear_model.LinearRegression()
lr_model2.fit(X_train, y_train)
lr_model2.score(X_test, y_test)

In [ ]:
y_df2 = pd.DataFrame(y_test)
y_df2['pred'] = lr_model2.predict(X_test)
y_df2['err'] = y_df2.pred - y_df2.close
(
y_df2
 #   .iloc[-50:]
    .plot(figsize=(14,10))
)

In [ ]:
# yellow brick version
fig, ax = plt.subplots(figsize=(10, 10))
err_viz2 = PredictionError(lr_model2)
err_viz2.score(X_test, y_test)
err_viz2.poof()

## Exercise: Regression with Train/Test Split

Split the data into test and training data. What is the score on the test data?

## Visualize Errors with Residual Plots
A residual is the difference between the prediction and the actual.
If we plot predicted value against residuals, we should get a random
distribution. If not, a different model more be better given the data.

In [ ]:
def residual_plot(model, X_train, y_train, X_test, y_test):
    fig = plt.figure(figsize=(14,10))
    ax = plt.subplot(111)
    plt.scatter(model.predict(X_train), 
                model.predict(X_train) - y_train, 
                c='b', alpha=.3,
                label='train')
    plt.scatter(model.predict(X_test), 
                model.predict(X_test) - y_test, 
                color='green', alpha=.3,
                label='test')
    plt.title('Residual Plot - Train (blue), Test (green)')
    plt.ylabel('Residual')
    ax.legend()
    


In [ ]:
residual_plot(lr_model2, X_train, y_train, X_test, y_test)

In [ ]:
# Yellowbrick version
fig, ax = plt.subplots(figsize=(10, 10))
res_viz = ResidualsPlot(lr_model2)
res_viz.fit(X_train, y_train)
res_viz.score(X_test, y_test)
res_viz.poof()

## Exercise - Residual Plot
Make a residual plot of your test and train data

## SVM, Random Forest, & Huber  - Other models

In [ ]:
# drop last row because we don't know what future is

X = (bit_df
         .drop(['close'], axis=1)
         .iloc[:-1])
y = bit_df.close.shift(-1).iloc[:-1]
cols = X.columns

ss = preprocessing.StandardScaler()
ss.fit(X)
X = ss.transform(X)
X = pd.DataFrame(X, columns=cols)

X_train, X_test, y_train, y_test = model_selection.\
    train_test_split(X, y, test_size=.3, random_state=42)
    
svm_model = svm.SVR(kernel='linear')
svm_model.fit(X_train, y_train)
svm_model.score(X_test, y_test)    

In [ ]:
def train_reg_model(model, df):
    # drop last row because we don't know what future is

    X = (df
             .drop(['close'], axis=1)
             .iloc[:-1])
    y = df.close.shift(-1).iloc[:-1]
    cols = X.columns

    ss = preprocessing.StandardScaler()
    ss.fit(X)
    X = ss.transform(X)
    X = pd.DataFrame(X, columns=cols)

    X_train, X_test, y_train, y_test = model_selection.\
        train_test_split(X, y, test_size=.3, random_state=42)

    #svm_model = svm.SVR(kernel='linear')
    model.fit(X_train, y_train)
    return model.score(X_test, y_test), X_test, y_test, X_train, y_train    
    
rf_reg = ensemble.RandomForestRegressor() 
score, X_test, y_test, X_train, y_train = train_reg_model(rf_reg, bit_df)
print(score)    

In [ ]:
def error_plot(X_test, y_test, model):
    y_df3 = pd.DataFrame(y_test)
    y_df3['pred'] = model.predict(X_test)
    y_df3['err'] = y_df3.pred - y_df3.close
    (
    y_df3
     #   .iloc[-50:]
        .plot(figsize=(14,10))
    )
error_plot(X_test, y_test, rf_reg)

In [ ]:
# yellow brick version
fig, ax = plt.subplots(figsize=(10, 10))
err_viz3 = PredictionError(rf_reg)
err_viz3.score(X_test, y_test)
err_viz3.poof()

In [ ]:

    
residual_plot(rf_reg, X_train, y_train, X_test, y_test)

In [ ]:
huber_reg = linear_model.HuberRegressor()
huber_reg.fit(X_train, y_train)
huber_reg.score(X_test, y_test)

In [ ]:
error_plot(X_test, y_test, huber_reg)

In [ ]:
# yellow brick version
fig, ax = plt.subplots(figsize=(10, 10))
err_viz4 = PredictionError(huber_reg)
err_viz4.score(X_test, y_test)
err_viz4.poof()

In [ ]:
residual_plot(huber_reg, X_train, y_train, X_test, y_test)

In [ ]:
huber_reg

## Exercise:

Try using another model (RandomForestRegressor or SVM)